[__Note: Charts look better on desktop but we realize that 75% of our users are on mobile. We're working on it and should have a solution soon. The charts are really cool so feel free to fire up the ol' tower.__]
# POTD (Pick of the Day)
#### There is no Pick of the Day today.

Our models couldn't agree on players for today, so it apparently isn't a good day to make a pick!

Here were the model outputs:

### Random Forest:

In [2]:
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go


rf = pd.DataFrame([['Mitch Moreland', 0.275], 
                   ['Jose Altuve', 0.272], 
                   ['Mike Trout', 0.27]],
columns= ['Player', 'Probability']
)

rf_table = ff.create_table(rf)
py.iplot(rf_table, filename='potd-rf-4-19-22')

### Logistic Regression:

In [3]:
lr = pd.DataFrame([['Jose Altuve', 0.301], 
                   ['Kole Calhoun', 0.284], 
                   ['Mitch Moreland', 0.277]],
columns= ['Player', 'Probability']
)

lr_table = ff.create_table(lr)
py.iplot(lr_table, filename='potd-lr-4-19-22')

Moreland was the only batter that qualified in the random forest while Altuve was the only one who qualified in Logistic Regression. Since our models don't agree on qualified picks, today is a no pick day. But even if they had agreed on picks, it possibly would've been one anyway due to the extremely low probabilities that the models are assigning players for the day.

Altuve is the only batter who would've passed our average hit probability threshold if our random forest liked him more over Moreland. However, he would've passed _barely_, which isn't a huge endorsement considering our probability threshold is quite low.

The remainder of the post will go over the odds of winning beat the streak.

### Beating the Streak
So how hard is it to actually beat the streak? Well let's imagine our program like a *player* and see what the odds would need to be for this player to get 57 hits in a row. Let's imagine that our hypothetical *player* has a 0.250 batting average. This means that for every at bat, this player has a 25% chance of getting a hit. If our player had 1 at bat per game, this means that this player has a 25% chance of getting a hit in a game. The chance of this player beating the streak would be __*0.250 ^ 57 = 4.8e-35*__. That's about the same probability as winning the lottery (1 in 13 million), and then consequently finding a single atom in an adult human (~10^27). In other words, impossible.

#### Multiple at-bats per game
But we would expect our player to get multiple at-bats per game, on average a number closer to 4. This means we can just multiply the batting average by 4 right? *0.250 x 4 = 1.000??*

Unfortunately stats don't work like that. The easiest way of seeing this is that the chance of __not__ getting a hit is the same as 4 consecutive at-bats without a hit, or *0.75 ^ 4 = 0.316* . To figure out what the probability of getting a hit is, we can just take this number and subtract it from 1. Here is the formula for probability of a single game with a hit:

__P (hit) = 1 - (1 - BA)^(AB)__

So for our hypothetical player with a BA of 0.250, the number becomes __*P (hit) = 1 - (1 - 0.250)^(4) = 0.683*__. Extra at-bats can help a __ton__ as well. If our player had 5 at bats instead, then __*P (hit) = 1 - (1 - 0.250)^(5) = 0.763*__, which is a pretty good jump in single game probability. This means it helps to have batters early in the lineup, and extra innings are also great. The following chart shows how single game probabilities change with BA and scale with number of at-bats. 

(We also include the python code that generated all the numbers so you can generate charts like these yourself)

In [15]:
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py

# line out batting averages as strings that we can use as labels later
BA_tested = ['0.100', '0.150', '0.200', '0.250', '0.300', '0.350', '0.400', '0.450']

# test AB's from 1 to 20 since this is just a thought experiment (plus it's not IMPOSSIBLE, __LINK__)
AB_tested = np.asarray(range(1,21))

# set up a dictionary to hold result arrays and populate
arrays = {}
data = []

for BA_string in BA_tested:
    BA_float = float(BA_string)
    arrays[BA_string] = 1 - np.power((1 - BA_float), (AB_tested))

    trace = go.Scatter(
        x = AB_tested,
        y = arrays[BA_string],
        mode = 'lines+markers',
        name = BA_string
    )
    
    data.append(trace)
    

layout = go.Layout(
    showlegend=True,
    title='Single Game Probability',
    yaxis=dict(
        title='Single Game Probability'
    ),
    xaxis=dict(
        title='Number of At-Bats'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Single-Game-Probability')

As you can see from the chart, at 4 AB's, our 0.250 hitter has a 68% chance of getting a hit. If this player were a 0.300 hitter, then this becomes 76%. If the player were a 0.350 hitter, then the chance becomes 82%. Now if we look at the single season record for batting average (https://www.baseball-reference.com/leaders/batting_avg_season.shtml), which is around 0.450, then the probability of getting a hit in 4 AB's is around 91%. If the player with the best season average BA were to get 5 AB's per game, then this probability becomes 95%.

### Probability of getting a 57 game streak
Let's take our hypothetical player and see how they fare when looking at 57 games in a row. Here we look at the probability of our 0.450 hitter beating the streak if he has 4 at-bats and 5 at-bats:

__P (BTS, 4 ABS, 0.450 BA) = 0.91 ^ 57 = 0.00462__

__P (BTS, 5 ABS, 0.450 BA) = 0.95 ^ 57 = 0.0537__

WOW! Just getting one more AB per game makes a player more than __10x__ more likely to get 57 hits in a row. The probability goes from 0.46% to 5.4%. Unfortunately we probably can't count on that, instead we'll assume 4 at-bats and see how sensitive the number is to batting average. Let's take a look at the player with the the best batting average in MLB right now (http://www.espn.com/mlb/stats/batting), Cody Bellinger from the Dodgers. He has a 0.424 batting average, so he has the following chance of beating the streak:

__ P(BTS, 4 ABS, 0.424 BA) = (1 - (1 - 0.424)^4) ^ 57 = 0.00129__

This is crazy! This means that a 0.450 player beating the streak is __3.6x more likely__ to beat the streak than a 0.424 player, meaning a little increase in batting average can go a LONG WAY.

The following chart shows batting average and the probability of beating the streak, and should illustrate how we really need players with good batting average.

In [21]:
BA_tested = np.asarray(range(1,101)) / 100
single_game_probabilities = 1 - np.power((1 - BA_tested), 4)
bts_probabilities = np.power(single_game_probabilities, 57)

trace = go.Scatter(
        x = BA_tested,
        y = bts_probabilities,
        mode = 'lines+markers'
    )

layout = go.Layout(
    showlegend=False,
    title='BTS Probability',
    yaxis=dict(
        title='BTS Probability'
    ),
    xaxis=dict(
        title='Batting Average'
    )
)

data=[trace]

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='BTS-Probability')

You can see from the chart that the probability is pretty much zero until we reach a batting average of around 0.500, which no player has ever done in a season. But after that it scales up pretty quickly.

### Random Forest and Logistic Regression
If we think of our two models as players, what would their equivalent batting average be? Last time we updated you on performance (https://seaminglyaccurate.io/potd-2019-04-21.html), both models were around 82% accurate. This is the equivalent of having a player with about a 0.350 batting average. If we combine the two models, though, the strategy is 91% accurate. This is the equivalent of having the player with the best batting average ever playing for us every day!

In order to have a 50 / 50 chance of beating the streak, we need *equivalent players* with 0.670 batting averages or higher. In order to have an __80%__ chance of beating the streak, we need *equivalent players* with 0.750 batting averages. The 0.750 batting average gives us a much better chance of beating the streak, so we think that's a good goal.

The good thing is that batting averages take into account all the days that player plays in a season. What we are trying to do is find good matchups and other pieces of data that will help us find the conditions for picking a player to get much higher batting averages than their season average. Our goal with feeding these models additional data is to find pockets of performance where the season long batting average doesn't tell the whole story. Some of this might include:

* Ballpark
* Pitcher matchup BA
* ABs against current pitcher

Stay tuned as we continue to track our performance, and hopefully improve our performance until we have a .750 hitter out there playing for us!